## Conditional LSTM - MimicShift
CIC2017 Wed (HULK)

In [1]:
from mimic import utils
from mimic.advgen_mimic import *

import tensorflow as tf
import scipy.sparse as sp
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score
import time
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import pickle
import yaml


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx

/home/wesleyjtann/miniconda3/envs/ddosgen/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


### Load data

In [3]:
## Select dataset
dataset_dir = '3_testmimic_caida07' #'3_testmimic_cicFriday' #'3_testmimic_cicWed' #
artefact_dir = 'artefact' 

if dataset_dir == "3_testmimic_cicWed":
    ## cicWed
    '''Load config file'''
    dataset = 'hulk'
    with open('../'+dataset_dir+'/light_config_'+dataset+'.yaml', "r") as ymlfile:
        config = yaml.load(ymlfile)
    '''Load normal traffic data'''
    # df_normal = pd.read_csv("../"+ dataset_dir + "/ids_test2/artefact_" + dataset + "/N1.csv")
    ## new normal = atk data
    df_normal = pd.read_csv("../"+ dataset_dir + "/ids_test2/artefact/" + "A1_full.csv")
else:
    ## cicFriday and caida07
    '''Load config file'''
    with open('../'+dataset_dir+'/light_config.yaml', "r") as ymlfile:
        config = yaml.load(ymlfile)
    '''Load normal traffic data'''
    # df_normal = pd.read_csv("../"+ dataset_dir + "/ids_test2/artefact/N1.csv")
    ## new normal = atk data
    df_normal = pd.read_csv("../"+ dataset_dir + "/ids_test2/artefact/" + "A1_full.csv")

/home/wesleyjtann/miniconda3/envs/ddosgen/lib/python3.6/site-packages/ipykernel_launcher.py:19: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [4]:
df_normal.head()

,Source IP,Dest IP,Protocols,Info,Input,Histo
0,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."
1,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."
2,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."
3,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."
4,192.120.148.227,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x0200, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x0200, seq=1280/5, tt..."


In [5]:
def getTokenizer(df) :
    ### Dictionary for Normal ###
    tokenizer = Tokenizer(filters='', split='<sep>', oov_token=None, lower=True, num_words=config['variablesHash']['inputHashRange']) #oov_token='<OTHERS>'
    tokenizer.fit_on_texts(df['Input'].values)

    tokenizer.fit_on_texts(['<SOS>'])
    tokenizer.fit_on_texts(['<EOS>'])
    return tokenizer

def createGeneratorData(df, tokenizer, max_len) :
    #Prepare training for normal model
    x = []
    y = []

    for seq in df['Input']:
        x_windows, y_windows = prepare_sentence(seq, max_len, tokenizer)
        x += x_windows
        y += y_windows
    x = np.array(x)
    y = np.array(y)  # The word <PAD> does not constitute a class

#     x.shape = [len(x), max_len + 1, 1]
#     y.shape = [len(y), max_len + 1, 1]
    x.shape = [len(x), max_len, 1]
    y.shape = [len(y), max_len , 1]

    return x, y

def prepare_sentence(seq, maxlen, tokenizer):
    # Pads seq and slides windows
    seq = seq[:maxlen]
    seqX = seq #np.append(tokenizer.word_index['<sos>'], seq)
    seqY = seq #np.append(seq, tokenizer.word_index['<eos>'])

    x= pad_sequences([seqX],
        maxlen=maxlen, #maxlen+1,
        padding='post')[0]  # Pads before each sequence

    y= pad_sequences([seqY],
        maxlen=maxlen, #maxlen+1,
        padding='post')[0]  # Pads before each sequence

    return [x], [y]

In [6]:
max_len = config['SEQUENCELENGTH']
tokenizer_normal = getTokenizer(df_normal)
df_normal_embedded = df_normal.copy()
df_normal_embedded['Input'] = tokenizer_normal.texts_to_sequences(df_normal['Input'].values)

x_normal, y_normal = createGeneratorData(df_normal_embedded, tokenizer_normal, max_len)

In [7]:
def score_matrix_from_random_walks(random_walks, N, symmetric=True):
    """
    Compute the transition scores, i.e. how often a transition occurs, for all node pairs from
    the random walks provided.
    Parameters
    ----------
    random_walks: np.array of shape (n_walks, rw_len, N)
                  The input random walks to count the transitions in.
    N: int
       The number of nodes
    symmetric: bool, default: True
               Whether to symmetrize the resulting scores matrix.

    Returns
    -------
    scores_matrix: sparse matrix, shape (N, N)
                   Matrix whose entries (i,j) correspond to the number of times a transition from node i to j was
                   observed in the input random walks.

    """

    random_walks = np.array(random_walks)
    bigrams = np.array(list(zip(random_walks[:, :-1], random_walks[:, 1:])))
    bigrams = np.transpose(bigrams, [0, 2, 1])
    bigrams = bigrams.reshape([-1, 2])
    if symmetric:
        bigrams = np.row_stack((bigrams, bigrams[:, ::-1]))

    mat = sp.coo_matrix((np.ones(bigrams.shape[0]), (bigrams[:, 0], bigrams[:, 1])),
                        shape=[N, N])
    return mat

def graph_from_scores(scores, n_edges):
    """
    Assemble a symmetric binary graph from the input score matrix. Ensures that there will be no singleton nodes.
    See the paper for details.

    Parameters
    ----------
    scores: np.array of shape (N,N)
            The input transition scores.
    n_edges: int
             The desired number of edges in the target graph.

    Returns
    -------
    target_g: symmettic binary sparse matrix of shape (N,N)
              The assembled graph.

    """

    if  len(scores.nonzero()[0]) < n_edges:
        return symmetric(scores) > 0

    target_g = np.zeros(scores.shape) # initialize target graph
    scores_int = scores.toarray().copy() # internal copy of the scores matrix
    scores_int[np.diag_indices_from(scores_int)] = 0  # set diagonal to zero
    degrees_int = scores_int.sum(0)   # The row sum over the scores.

    N = scores.shape[0]

    for n in np.random.choice(N, replace=False, size=N): # Iterate the nodes in random order

        row = scores_int[n,:].copy()
        if row.sum() == 0:
            continue

        probs = row / row.sum()

        target = np.random.choice(N, p=probs)
        target_g[n, target] = 1
        target_g[target, n] = 1


    diff = np.round((n_edges - target_g.sum())/2)
    if diff > 0:

        triu = np.triu(scores_int)
        triu[target_g > 0] = 0
        triu[np.diag_indices_from(scores_int)] = 0
        triu = triu / triu.sum()

        triu_ixs = np.triu_indices_from(scores_int)
        extra_edges = np.random.choice(triu_ixs[0].shape[0], replace=False, p=triu[triu_ixs], size=int(diff))

        target_g[(triu_ixs[0][extra_edges], triu_ixs[1][extra_edges])] = 1
        target_g[(triu_ixs[1][extra_edges], triu_ixs[0][extra_edges])] = 1

    target_g = symmetric(target_g)
    return target_g

def symmetric(directed_adjacency, clip_to_one=True):
    """
    Symmetrize the input adjacency matrix.
    Parameters
    ----------
    directed_adjacency: sparse matrix or np.array of shape (N,N)
                        Input adjacency matrix.
    clip_to_one: bool, default: True
                 Whether the output should be binarized (i.e. clipped to 1)

    Returns
    -------
    A_symmetric: sparse matrix or np.array of the same shape as the input
                 Symmetrized adjacency matrix.

    """

    A_symmetric = directed_adjacency + directed_adjacency.T
    if clip_to_one:
        A_symmetric[A_symmetric > 1] = 1
    return A_symmetric

In [8]:
num_nodes = len(np.unique(x_normal))
num_nodes

250

In [9]:
# Compute score matrix
# gr = utils.score_matrix_from_random_walks(np.array(smpls).reshape([-1, self.rw_len]), self.N)
gr = score_matrix_from_random_walks(np.array(x_normal).reshape([-1, max_len]), num_nodes)
gr = gr.tocsr() # shape=(N, N)                
print("gr: ", gr.shape)

# max number of edges
max_edges = (gr.shape[0] * (gr.shape[0]-1) )/2 #285390.0

## Assemble a graph from the score matrix
# n_edges = int((num_nodes)*2)
n_edges = int(max_edges/4)
_graph = graph_from_scores(gr, n_edges) # shape=(N, N)
print("_graph: ", _graph)

## Preparing data
Adjtraining = sp.csr_matrix(_graph, dtype='float64')
_A_obs = Adjtraining
_A_obs = _A_obs + _A_obs.T # 
_A_obs[_A_obs > 1] = 1 # Max value of 1 

""" Reduce input graph to a subgraph where only the nodes in largest n_components are kept. """ 
lcc = utils.largest_connected_components(_A_obs) # 
_A_obs = _A_obs[lcc,:][:,lcc] # 
_N = _A_obs.shape[0] # 

gr:  (250, 250)
_graph:  [[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Selecting 1 largest connected components


### """ Set the list of conditions """

In [10]:
""" Set the list of conditions """
# # randomly assigned conditions to num_classes
# num_classes = 3
# lcc_condlist = np.concatenate((np.arange(_N).reshape(_N,1), np.random.randint(num_classes, size=_N).reshape(_N,1)), axis=1) 
num_classes = 3
cond = np.genfromtxt ('../'+dataset_dir+'/packetlen_cond.csv', delimiter=",") #1_testmimic_cicFriday
# cond = np.append(cond, np.zeros(_N-len(cond)))
lcc_condlist = np.concatenate((np.arange(_N).reshape(_N,1), cond.reshape(_N,1)), axis=1) 

## Create our generative model 

In [11]:
#### Separate the edges into train, test, validation
val_share = 0.1
test_share = 0.05
seed = 2021 #  
"""
Split the edges of the adjacency matrix into train, validation and test edges and randomly samples equal amount of validation and test non-edges. 
"""
train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False) 

## EGGen
train_graph = sp.coo_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1]))).tocsr()
assert (train_graph.toarray() == train_graph.toarray().T).all()


#### Parameters
""" Adjustable parameters for training. """ 
# setting GPU id 
gpu_id = 0 # 1 # 
# setting the number of nodes
_N = _A_obs.shape[0]
# setting the length of random walks
rw_len = max_len #20 #
# setting the training data batch size
batch_size = 128 #
# getting the number of departments
n_conds=len(np.unique(lcc_condlist[:,1]))
print("n_conds: ", n_conds)
sample_batch = 128 #
# log_num = 0 #

walker = utils.RandomWalker(train_graph, lcc_condlist, rw_len, p=1, q=1, batch_size=batch_size, sample_batch=sample_batch)

n_conds:  3


In [12]:
# ## Create our generative model 
l2_gen=1e-7; l2_disc=5e-5 #1e-4 
# gencond_lay=[10]; gen_lay=[50]; disc_lay=[40] # try gen_lay=[40]; disc_lay=[30]
gencond_lay=[10]; gen_lay=[40]; disc_lay=[30] # try gen_lay=[40]; disc_lay=[30]
lr_gencond=0.01; lr_gen=0.0002; lr_disc=0.0002
gencond_iters=1; gen_iters=3; disc_iters=1
discWdown_size=128; genWdown_size=128 

eggen = EGGen(_N,
rw_len,
walk_generator=walker,
n_conds=n_conds,
condgenerator_layers=gencond_lay,
generator_layers=gen_lay,
discriminator_layers=disc_lay,
W_down_discriminator_size=discWdown_size,
W_down_generator_size=genWdown_size,
batch_size=batch_size,
sample_batch=sample_batch,
condition_dim=n_conds,
gencond_iters=gencond_iters,
gen_iters=gen_iters,
disc_iters=disc_iters,
wasserstein_penalty=10, 
l2_penalty_generator=l2_gen,
l2_penalty_discriminator=l2_disc,
lr_gencond=lr_gencond,
lr_gen=lr_gen,
lr_disc=lr_disc,
noise_dim=16, #
noise_type="Gaussian", #
temp_start=5.0, #
min_temperature=0.5,
temperature_decay=1-5e-5,
seed=15, #seed, #
use_gumbel=True,
legacy_generator=False,
gpu_id=gpu_id,
plot_show=False
)


intermediate:  Tensor("Generator/Generator.int_1/Tanh:0", shape=(128, 40), dtype=float32)
h:  Tensor("Generator/Generator.h_1/Tanh:0", shape=(128, 40), dtype=float32)
c:  Tensor("Generator/Generator.c_1/Tanh:0", shape=(128, 40), dtype=float32)
Generator initial_states:  1
Initial cond:  Tensor("Generator/unstack:0", shape=(128, 3), dtype=float32)


## Training the model 

## Load trained model 

In [13]:
""" ===================================================================================== """
## RANDOM COND: cicWed: model_best_0cicWed; cicFriday: model_best_1cicFriday; caida07: model_best_2caida07
## PktLen COND: cicWed: model_best_0; cicFriday: model_best_1; caida07: model_best_2
model_name = "model_best_2" # 0 (wed), 1(fri), 2(caida)
print("Model: ", model_name)
saver = tf.train.Saver()
saver.restore(eggen.session, "snapshots_mimic/" + model_name + ".ckpt") # 

Model:  model_best_2
INFO:tensorflow:Restoring parameters from snapshots_mimic/model_best_2.ckpt


## Generating traffic

In [14]:
sample_many, explain_conds = eggen.generate_discrete(1000, conds=True, rw_len=rw_len, reuse=True) 

intermediate:  Tensor("Generator_1/Generator.int_1/Tanh:0", shape=(1000, 40), dtype=float32)
h:  Tensor("Generator_1/Generator.h_1/Tanh:0", shape=(1000, 40), dtype=float32)
c:  Tensor("Generator_1/Generator.c_1/Tanh:0", shape=(1000, 40), dtype=float32)
Generator initial_states:  1
Initial cond:  Tensor("Generator_1/unstack:0", shape=(1000, 3), dtype=float32)


In [15]:
import time
t0 = time.time()

num_paths = 10 ##15 
samples = []
for _ in range(num_paths): 
    if (_+1) % round(num_paths/3) == 0:
        print(_+1)
    samples.append(sample_many.eval({eggen.tau: 0.5}))
    
t1 = time.time()
print("total time: ", t1-t0)

3
6
9
total time:  3.8002374172210693


In [16]:
# Generated fake traffic sequences
print(len(samples))
samples[0].shape

10


(1000, 16)

In [17]:
gentraffic = np.reshape(samples, (-1, rw_len))
gentraffic # (num_requests, rw_len)

array([[175, 110,  37, ..., 218,   8,  36],
       [201, 185,  74, ..., 175, 159,  95],
       [ 39, 170, 230, ...,  13, 181, 175],
       ...,
       [ 56,  10, 124, ..., 196,  19,  95],
       [ 90,  60, 133, ..., 177, 148,  72],
       [144,   6, 142, ..., 240,  15,  95]])

In [18]:
gentraffic.shape

(10000, 16)

In [19]:
# Creating test data
df_tmp = pd.DataFrame(data=gentraffic.astype('str'))

df_gentest = pd.DataFrame()
df_gentest["Source IP"] = ""
df_gentest["Dest IP"] = ""
df_gentest['Input'] = df_tmp.agg(lambda x: "<SEP>".join(x), axis=1)
df_gentest["Source IP"] = "999.16.30.0"
df_gentest["Dest IP"] = "999.16.30.1"
df_gentest["Attack"] = ""
# True attacks
df_gentest['Attack']=1

print("length df_gentest: ",  len(df_gentest)) 
df_gentest

length df_gentest:  10000


,Source IP,Dest IP,Input,Attack
0,999.16.30.0,999.16.30.1,175<SEP>110<SEP>37<SEP>175<SEP>72<SEP>139<SEP>...,1
1,999.16.30.0,999.16.30.1,201<SEP>185<SEP>74<SEP>139<SEP>19<SEP>144<SEP>...,1
2,999.16.30.0,999.16.30.1,39<SEP>170<SEP>230<SEP>33<SEP>177<SEP>203<SEP>...,1
3,999.16.30.0,999.16.30.1,175<SEP>182<SEP>207<SEP>11<SEP>194<SEP>8<SEP>1...,1
4,999.16.30.0,999.16.30.1,91<SEP>21<SEP>98<SEP>229<SEP>44<SEP>165<SEP>22...,1
5,999.16.30.0,999.16.30.1,48<SEP>8<SEP>106<SEP>197<SEP>21<SEP>27<SEP>65<...,1
6,999.16.30.0,999.16.30.1,20<SEP>19<SEP>104<SEP>226<SEP>206<SEP>36<SEP>1...,1
7,999.16.30.0,999.16.30.1,153<SEP>124<SEP>43<SEP>10<SEP>50<SEP>25<SEP>15...,1
8,999.16.30.0,999.16.30.1,87<SEP>6<SEP>111<SEP>241<SEP>217<SEP>8<SEP>36<...,1
9,999.16.30.0,999.16.30.1,14<SEP>123<SEP>153<SEP>34<SEP>45<SEP>50<SEP>34...,1


### ============ TRY! Save generated traffic for training (Not USED) ============


In [21]:
## Retrieve the original test set
# df_test_original = pd.read_csv('../'+dataset_dir+'/ids_test2/artefact_hulk/TEST.csv') # cicWed
# df_test_original = df_test_original.drop(columns=['Protocols', 'Info']) #
# 
df_test_original = pd.read_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/TESTnorm.csv') #artefact_mimic0_seqlen200hash750 #cicFriday
df_test_original = df_test_original.drop(columns=['Protocols', 'Info', 'Histo']) #, 'Histo'

In [22]:
print(len(df_test_original))
df_test_original

2263


,Source IP,Dest IP,Input,Attack
0,192.95.27.190,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
1,192.120.148.227,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
2,40.75.89.172,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
3,40.75.89.172,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
4,40.75.89.172,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
5,51.173.229.255,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
6,202.1.175.252,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
7,51.173.229.255,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0
8,198.241.152.229,71.126.222.64,25<SEP>115<SEP>100<SEP>249<SEP>38<SEP>245<SEP>...,0
9,40.75.89.172,71.126.222.64,2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,0


In [23]:
# Concatenate and shuffle new test set
print(len(df_gentest))
df_gentest = pd.concat([df_gentest, df_test_original]).sample(frac=1).reset_index(drop=True)
len(df_gentest)

10000


12263

In [24]:
## Saving df_gentest to csv 
df_gentest.to_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/GENTEST.csv', index=False) 
# df_gentest.to_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/GENTEST0.csv', index=False)

## Controlled generation - Markov chain

In [25]:
unique_conds, conds_counts = np.unique(lcc_condlist[:,1], return_counts=True)
print(unique_conds)
print(conds_counts)

dept_list = unique_conds
dept_list

[0. 1. 2.]
[178   3  69]


array([0., 1., 2.])

In [26]:
# Initial state distribution
init_distr = [0.9, .05, .05]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.9,0.1,0.0],[0.1,0.6,0.3],[0.2,0.7,0.1]]

if sum(transitionMatrix[0])+sum(transitionMatrix[1])+sum(transitionMatrix[1]) != 3:
    print("Somewhere, something went wrong. Transition matrix, perhaps?")
else: print("All is gonna be okay, you should move on!! ;)")

All is gonna be okay, you should move on!! ;)


In [27]:
def generate_controltrans(init_distr, transitionMatrix, dept_list, n_samples=10000, rw_len=16):
        
    shadow_walks = np.empty([n_samples, rw_len], dtype=int)

    for sample_shadowwalk in range(n_samples):
        shadow_walk = [] #np.array([], dtype=int)
        walk_start = np.random.choice(dept_list, p=init_distr)

        step = walk_start

        for _ in range(rw_len-1):
        #     print(_)
            if _ == 0:
                shadow_walk.append(walk_start)

            if step == dept_list[0]:
                step = np.random.choice(dept_list,replace=True,p=transitionMatrix[0])
#                 p=intra_dept_prob; q=inter_dept_prob; r=inter_dept_prob
            elif step == dept_list[1]:
                step = np.random.choice(dept_list,replace=True,p=transitionMatrix[1])
#                 p=inter_dept_prob; q=intra_dept_prob; r=inter_dept_prob
            else:
                step = np.random.choice(dept_list,replace=True,p=transitionMatrix[2])
#                 p=inter_dept_prob; q=inter_dept_prob; r=intra_dept_prob

#             step = np.random.choice(dept_list, p=[p, q, r])
            shadow_walk.append(step)

    #     print(shadow_walk)

        shadow_walks[sample_shadowwalk] = shadow_walk

    return np.array(shadow_walks)

In [28]:
def controlled_generation(init_distr, transitionMatrix, multiplier, controlgen_iter, control_dept_list, condition_dim, n_samples=1000, rw_len=16):
#     control_conds_discrete = np.random.choice(control_dept_list, size=[n_samples,rw_len], p=[p, q, r])
    control_conds_discrete = generate_controltrans(init_distr, transitionMatrix, control_dept_list, n_samples, rw_len)
    conds = tf.one_hot(control_conds_discrete, condition_dim) # (n_samples, rw_len, n_conds)
    control_samplemany, control_conds = eggen.generate_discrete(n_samples, conds=conds, rw_len=rw_len, reuse=True)
    
    start_time = time.time()
    control_samples = []
    for _ in range(controlgen_iter): 
        if (_+1) % 1000 == 0:
            print(_+1)
#             control_conds_discrete = np.random.choice(control_dept_list, size=[n_samples,rw_len], p=[p, q, r])
            control_conds_discrete = generate_controltrans(init_distr, transitionMatrix, control_dept_list, n_samples, rw_len)
            conds = tf.one_hot(control_conds_discrete, condition_dim) # (n_samples, rw_len, n_conds)
            control_samplemany, control_conds = eggen.generate_discrete(n_samples, conds=conds, rw_len=rw_len, reuse=True)
        control_samples.append(control_samplemany.eval({eggen.tau: 0.5}))
    print("--- %s seconds ---" % (time.time() - start_time))
    
    control_rws = np.array(control_samples).reshape([-1, rw_len])
    print("control_rws: ", control_rws.shape)
#     control_scrmatrix = utils.score_matrix_from_random_walks(control_rws, _N).tocsr()
    
#     """ Compute control graph statistics"""
#     control_graph = utils.graph_from_scores(control_scrmatrix,  _A_obs.sum()*multiplier)
    
#     print("EO: ", utils.edge_overlap(_A_obs.toarray(), control_graph)/_A_obs.sum()*multiplier)
    
#     G_control = nx.from_numpy_matrix(control_graph)

#     # Assign attribute to G_control
#     for row in lcc_condlist:
#         G_control.nodes[row[0]]['attr'] = row[1]
        
    return control_rws #G_control

In [29]:
## 3 classes
req_control1 = controlled_generation(init_distr=[0.9, 0.05, 0.05], 
                                   transitionMatrix=[[0.98,0.01,0.01],[0.1,0.6,0.3],[0.0,0.1,0.9]], 
                                   multiplier=1, controlgen_iter=num_paths, control_dept_list=dept_list, 
                                   condition_dim=n_conds, rw_len=rw_len)

req_control2 = controlled_generation(init_distr=[0.05, 0.05, 0.9], 
                                   transitionMatrix=[[0.9,0.1,0.0],[0.1,0.6,0.3],[0.03,0.02,0.95]], 
                                   multiplier=1, controlgen_iter=num_paths, control_dept_list=dept_list, 
                                   condition_dim=n_conds, rw_len=rw_len)

req_control3 = controlled_generation(init_distr=[0.05, 0.9, 0.05], 
                                   transitionMatrix=[[0.9,0.1,0.0],[0.1,0.7,0.2],[0.0,0.1,0.9]],
                                   multiplier=1, controlgen_iter=num_paths, control_dept_list=dept_list, 
                                   condition_dim=n_conds, rw_len=rw_len)

intermediate:  Tensor("Generator_2/Generator.int_1/Tanh:0", shape=(1000, 40), dtype=float32)
h:  Tensor("Generator_2/Generator.h_1/Tanh:0", shape=(1000, 40), dtype=float32)
c:  Tensor("Generator_2/Generator.c_1/Tanh:0", shape=(1000, 40), dtype=float32)
Generator initial_states:  1
Initial cond:  Tensor("Generator_2/unstack:0", shape=(1000, 3), dtype=float32)
--- 0.6895074844360352 seconds ---
control_rws:  (10000, 16)
intermediate:  Tensor("Generator_3/Generator.int_1/Tanh:0", shape=(1000, 40), dtype=float32)
h:  Tensor("Generator_3/Generator.h_1/Tanh:0", shape=(1000, 40), dtype=float32)
c:  Tensor("Generator_3/Generator.c_1/Tanh:0", shape=(1000, 40), dtype=float32)
Generator initial_states:  1
Initial cond:  Tensor("Generator_3/unstack:0", shape=(1000, 3), dtype=float32)
--- 0.6178092956542969 seconds ---
control_rws:  (10000, 16)
intermediate:  Tensor("Generator_4/Generator.int_1/Tanh:0", shape=(1000, 40), dtype=float32)
h:  Tensor("Generator_4/Generator.h_1/Tanh:0", shape=(1000, 40)

In [30]:
print(req_control1.shape)

(10000, 16)


In [31]:
print(req_control1.shape)

# Creating test data
df_tmp = pd.DataFrame(data=req_control1.astype('str'))

df_shiftgen_test1 = pd.DataFrame()
df_shiftgen_test1["Source IP"] = ""
df_shiftgen_test1["Dest IP"] = ""
df_shiftgen_test1['Input'] = df_tmp.agg(lambda x: "<SEP>".join(x), axis=1)
df_shiftgen_test1["Source IP"] = "999.16.30.0"
df_shiftgen_test1["Dest IP"] = "999.16.30.1"

# add original test and shuffle
df_shiftgen_test1 = pd.concat([df_shiftgen_test1, df_test_original]).sample(frac=1).reset_index(drop=True)

## Saving df_gentest to csv 
# df_shiftgen_test1.to_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/ShiftGENTEST1.csv', index=False)

(10000, 16)


/home/wesleyjtann/miniconda3/envs/ddosgen/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [32]:
print(req_control2.shape)

# Creating test data
df_tmp = pd.DataFrame(data=req_control2.astype('str'))

df_shiftgen_test2 = pd.DataFrame()
df_shiftgen_test2["Source IP"] = ""
df_shiftgen_test2["Dest IP"] = ""
df_shiftgen_test2['Input'] = df_tmp.agg(lambda x: "<SEP>".join(x), axis=1)
df_shiftgen_test2["Source IP"] = "999.16.30.0"
df_shiftgen_test2["Dest IP"] = "999.16.30.1"

# add original test and shuffle
df_shiftgen_test2 = pd.concat([df_shiftgen_test2, df_test_original]).sample(frac=1).reset_index(drop=True)

## Saving df_gentest to csv 
# df_shiftgen_test2.to_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/ShiftGENTEST2.csv', index=False)

(10000, 16)


/home/wesleyjtann/miniconda3/envs/ddosgen/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [33]:
print(req_control3.shape)

# Creating test data
df_tmp = pd.DataFrame(data=req_control3.astype('str'))

df_shiftgen_test3 = pd.DataFrame()
df_shiftgen_test3["Source IP"] = ""
df_shiftgen_test3["Dest IP"] = ""
df_shiftgen_test3['Input'] = df_tmp.agg(lambda x: "<SEP>".join(x), axis=1)
df_shiftgen_test3["Source IP"] = "999.16.30.0"
df_shiftgen_test3["Dest IP"] = "999.16.30.1"

# add original test and shuffle
df_shiftgen_test3 = pd.concat([df_shiftgen_test3, df_test_original]).sample(frac=1).reset_index(drop=True)

## Saving df_gentest to csv 
# df_shiftgen_test3.to_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/ShiftGENTEST3.csv', index=False)

(10000, 16)


/home/wesleyjtann/miniconda3/envs/ddosgen/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [34]:
df_A1full = pd.DataFrame()

In [35]:
# Randomly select 2000 requests from req_control1
df_shiftgen_test1
n = 2000 #1000 ##
if dataset_dir == "3_testmimic_cicFriday":
    train_range = 13
# elif dataset_dir == "3_testmimic_cicWed":
else:
    train_range = 12
# artefact_dir = 'artefact' #'hulk_test'

for i in range(0,train_range,3):
    df_shiftgen1 = df_shiftgen_test1.sample(n=n)
    df_A1full = pd.concat([df_A1full, df_shiftgen1])
#     df_shiftgen1.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_'+ str(i) + '.csv', index=False)
    df_shiftgen1.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_'+ str(i) + '.csv', index=False)
for i in range(1,train_range,3):
    df_shiftgen2 = df_shiftgen_test2.sample(n=n)
    df_A1full = pd.concat([df_A1full, df_shiftgen2])
#     df_shiftgen2.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_'+ str(i) + '.csv', index=False)
    df_shiftgen2.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_'+ str(i) + '.csv', index=False)
for i in range(2,train_range,3):
    df_shiftgen3 = df_shiftgen_test3.sample(n=n)
    df_A1full = pd.concat([df_A1full, df_shiftgen3])
#     df_shiftgen3.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_'+ str(i) + '.csv', index=False) 
    df_shiftgen3.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_'+ str(i) + '.csv', index=False)  

In [36]:
len(df_A1full) #.sample(frac=1).reset_index(drop=True)
# df_A1full.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_full.csv', index=False)      
df_A1full.to_csv('../'+dataset_dir+'/ids_test2/'+ artefact_dir +'/A1_full.csv', index=False)      

### TRY! save reqcontrol3 as test set (for CAIDA07 only)

In [ ]:
df_test_original = pd.read_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/TESTnorm.csv') #artefact_mimic0_seqlen200hash750 #cicFriday
df_test_original = df_test_original.drop(columns=['Protocols', 'Info', 'Histo']) #, 'Histo'

In [ ]:
# Creating test data
df_tmp = pd.DataFrame(data=req_control2.astype('str'))

df_shiftgen_test2 = pd.DataFrame()
df_shiftgen_test2["Source IP"] = ""
df_shiftgen_test2["Dest IP"] = ""
df_shiftgen_test2['Input'] = df_tmp.agg(lambda x: "<SEP>".join(x), axis=1)
df_shiftgen_test2["Source IP"] = "999.16.30.0"
df_shiftgen_test2["Dest IP"] = "999.16.30.1"

# True attacks
df_shiftgen_test2['Attack']=1

In [ ]:
len(df_test_original)

In [ ]:
# Concatenate and shuffle new test set
df_gentest = pd.concat([df_shiftgen_test2, df_test_original]).sample(frac=1).reset_index(drop=True)
len(df_gentest)

## Saving df_gentest to csv 
# df_gentest.to_csv('../'+dataset_dir+'/ids_test2/artefact_hulk/GENTEST.csv', index=False) 
df_gentest.to_csv('../'+dataset_dir+'/ids_test2/'+artefact_dir+'/GENTEST0.csv', index=False)